In [ ]:
%config IPCompleter.greedy = True  
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

from seaborn import countplot
from matplotlib.pyplot import figure, show

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import FactorRange, ColumnDataSource, TableColumn, DataTable, CustomJS, Legend, Range1d
from bokeh.models.widgets import Panel, Tabs, Select, CheckboxGroup
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.io import output_file, show
from bokeh.layouts import column, row, WidgetBox  
from bokeh.core.properties import value 

In [ ]:
#Database connectie
host = "mysql.nm-interactive.net"
port = 3306
user = "windesheim"
password = "pMjq357Kdee7Sx8C"
database = "windesheim"
client = "mysql"

#Installeer eerst Mysqlcient in Anaconda Navigator
engine = create_engine("{0}://{1}:{2}@{3}:{4}/{5}".format(client, user, password, host, port, database))

datasource = pd.read_sql_query("SELECT * FROM source WHERE KWALIFICATIENIVEAU = 4",engine)
wervingsgebieden = pd.read_sql_query("SELECT * FROM wervingsgebieden",engine)
datasource.info()
wervingsgebieden.info()

In [ ]:
# Functie voor het checken van de locatie en categorie, toegevoegd DB
def check_locatie(PostCode):
    test = wervingsgebieden.loc[(wervingsgebieden.PostcodeStart <= PostCode) & (wervingsgebieden.PostcodeEind >= PostCode), ['Locatie', 'Categorie']]
    if test['Locatie'].empty:
        r = pd.Series(['',''])
    else:
        r = pd.Series([test['Locatie'].iloc[0], test['Categorie'].iloc[0]])
    return r

#voeg kolommen toe aan datasource 
datasource[['Locatie', 'Categorie']] = datasource.apply(lambda row: check_locatie(row["PC4_LEERL"]), axis=1)

In [ ]:
#DB
dsc = datasource.copy()
#kolommen met AANT in de naam geven jaren weer
jaarCols = [col for col in dsc.columns if 'AANT' in col]
#dus wat overblijft zijn de overige kolommen
otherCols = [col for col in dsc.columns if not 'AANT' in col]
#haal aant weg van kolommen
jaarCleanCols = [str(x)[:4] for x in jaarCols]
#voeg de opgeschoonde kolommen samen met de originele
jaarDict = dict(zip(jaarCols, jaarCleanCols))
#hernoem de originele naar de opgeschoonde kolommen
dsc = dsc.rename(columns=jaarDict)
dscMelt = pd.melt(dsc, id_vars=otherCols, var_name="Jaar", value_name="Aantal")
df = dscMelt.copy()
df_totaalSBB = df.groupby(['SECTORUNIT SBB', 'Jaar'], as_index=False).sum()[['SECTORUNIT SBB', 'Jaar', 'Aantal']]

In [ ]:
#SvdB
dsc2 = datasource.copy()
dsc2 = dsc2.loc[dsc2['Locatie'] == 'Zwolle']
dsc2 = dsc2.rename(columns=jaarDict)
dsc2Melt = pd.melt(dsc2, id_vars=otherCols, var_name='Jaar', value_name='Aantal')
df_zwolle = dsc2Melt.copy()
df_totaalSBB_zwolle = df_zwolle.groupby(['SECTORUNIT SBB', 'Jaar'], as_index=False).sum()[['SECTORUNIT SBB', 'Jaar', 'Aantal']]

In [ ]:
#SvdB
dsc3 = datasource.copy()
dsc3 = dsc3.loc[dsc3['Locatie'] == 'Flevoland']
dsc3 = dsc3.rename(columns=jaarDict)
dsc3Melt = pd.melt(dsc3, id_vars=otherCols, var_name='Jaar', value_name='Aantal')
df_flevoland = dsc3Melt.copy()
df_totaalSBB_flevoland = df_flevoland.groupby(['SECTORUNIT SBB', 'Jaar'], as_index=False).sum()[['SECTORUNIT SBB', 'Jaar', 'Aantal']]

In [ ]:
#SvdB, DB en RR
def make_document(doc):
    #Defineer brondata & kolommen
    source_totaal = ColumnDataSource(df_totaalSBB)
    original_source_totaal = ColumnDataSource(df_totaalSBB)

    kolommen_totaal = list(df_totaalSBB['Jaar'].unique())
    sectoren_totaal = list(df_totaalSBB['SECTORUNIT SBB'].unique())

    columns = [
    TableColumn(field="SECTORUNIT SBB", title="Sector"),
    TableColumn(field="Jaar", title="Jaar"),
    TableColumn(field="Aantal", title="Aantal"),
    ]

    source_totaal_zwolle = ColumnDataSource(df_totaalSBB_zwolle)
    original_source_totaal_zwolle = ColumnDataSource(df_totaalSBB_zwolle)

    kolommen_totaal_zwolle = list(df_totaalSBB_zwolle['Jaar'].unique())
    sectoren_totaal_zwolle = list(df_totaalSBB_zwolle['SECTORUNIT SBB'].unique())

    source_totaal_flevoland = ColumnDataSource(df_totaalSBB_flevoland)
    original_source_totaal_flevoland = ColumnDataSource(df_totaalSBB_flevoland)

    kolommen_totaal_flevoland = list(df_totaalSBB_flevoland['Jaar'].unique())
    sectoren_totaal_flevoland = list(df_totaalSBB_flevoland['SECTORUNIT SBB'].unique())

    # ook legenda kleuren palet moet gedefineerd worden
    colortable = ['#084594', '#2171b5', '#4292c6', '#6baed6', '#9ecae1']

    #maak data tabel
    data_table_totaal = DataTable(source=source_totaal, columns=columns)
    data_table_totaal_zwolle = DataTable(source=source_totaal_zwolle, columns=columns)
    data_table_totaal_flevoland = DataTable(source=source_totaal_flevoland, columns=columns)

    # callback code to be used by all the filter widgets
    combined_callback_code = """
    var data = source.data;
    var original_data = original_source.data;
    var sector = sector_select_obj.value;
    console.log("sector: " + sector);
    var jaar = jaar_select_obj.value;
    console.log("jaar: " + jaar);
    for (var key in original_data) {
        data[key] = [];
        for (var i = 0; i < original_data['SECTORUNIT SBB'].length; ++i) {
            if ((sector === "ALL" || original_data['SECTORUNIT SBB'][i] === sector) &&
                (jaar === "ALL" || original_data['Jaar'][i] === jaar)) {
                data[key].push(original_data[key][i]);
            }
        }
    }
    target_obj.change.emit();;
    source.change.emit();
    """

    # definieer filter widgets
    sector_list_totaal = ['ALL'] + df_totaalSBB['SECTORUNIT SBB'].unique().tolist()
    sector_select_totaal = Select(title="Sector:", value=sector_list_totaal[0], options=sector_list_totaal)
    jaar_list_totaal = ['ALL'] + df_totaalSBB['Jaar'].unique().tolist()
    jaar_select_totaal = Select(title="Jaar:", value=jaar_list_totaal[0], options=jaar_list_totaal)

    sector_list_totaal_zwolle = ['ALL'] + df_totaalSBB_zwolle['SECTORUNIT SBB'].unique().tolist()
    sector_select_totaal_zwolle = Select(title="Sector:", value=sector_list_totaal_zwolle[0], options=sector_list_totaal_zwolle)
    jaar_list_totaal_zwolle = ['ALL'] + df_totaalSBB_zwolle['Jaar'].unique().tolist()
    jaar_select_totaal_zwolle = Select(title="Jaar:", value=jaar_list_totaal_zwolle[0], options=jaar_list_totaal_zwolle)

    sector_list_totaal_flevoland = ['ALL'] + df_totaalSBB_flevoland['SECTORUNIT SBB'].unique().tolist()
    sector_select_totaal_flevoland = Select(title="Sector:", value=sector_list_totaal_flevoland[0], options=sector_list_totaal_flevoland)
    jaar_list_totaal_flevoland = ['ALL'] + df_totaalSBB_flevoland['Jaar'].unique().tolist()
    jaar_select_totaal_flevoland = Select(title="Jaar:", value=jaar_list_totaal_flevoland[0], options=jaar_list_totaal_flevoland)

    # now define the callback objects now that the filter widgets exist
    generic_callback_totaal = CustomJS(
    args=dict(source=source_totaal, 
              original_source=original_source_totaal, 
              sector_select_obj=sector_select_totaal, 
              jaar_select_obj=jaar_select_totaal, 
              target_obj=data_table_totaal),
    code=combined_callback_code
    )

    generic_callback_totaal_zwolle = CustomJS(
    args=dict(source=source_totaal_zwolle, 
              original_source=original_source_totaal_zwolle, 
              sector_select_obj=sector_select_totaal_zwolle, 
              jaar_select_obj=jaar_select_totaal_zwolle, 
              target_obj=data_table_totaal_zwolle),
    code=combined_callback_code
    )

    generic_callback_totaal_flevoland = CustomJS(
    args=dict(source=source_totaal_flevoland,
              original_source=original_source_totaal_flevoland,
              sector_select_obj=sector_select_totaal_flevoland,
              jaar_select_obj=jaar_select_totaal_flevoland,
              target_obj=data_table_totaal_flevoland),
    code=combined_callback_code
    )

    # voorbereiden dataset

    df_copyForStack_totaal =  dsc.copy()
    df_copyForStackGrouped_totaal = df_copyForStack_totaal.groupby(['SECTORUNIT SBB']).sum()[kolommen_totaal]

    df_copyForStack_totaal_zwolle = dsc2.copy()
    df_copyForStackGrouped_totaal_zwolle = df_copyForStack_totaal_zwolle.groupby(['SECTORUNIT SBB']).sum()[kolommen_totaal_zwolle]

    df_copyForStack_totaal_flevoland = dsc3.copy()
    df_copyForStackGrouped_totaal_flevoland = df_copyForStack_totaal_flevoland.groupby(['SECTORUNIT SBB']).sum()[kolommen_totaal_flevoland]

    # Groepeer voor volgorde hoog-laag (nu nog met bug)
    #df_copyForStackGrouped.sort_values(by=kolommen, ascending=False, inplace=True)

    # Zet om naar dictionary voor verwerking door grafiek
    df_stackedToDict_totaal = df_copyForStackGrouped_totaal.to_dict('list')
    df_stackedToDict_totaal['sectoren'] = sectoren_totaal

    # Maak er een CDS van
    source_grafiek_totaal= ColumnDataSource(df_stackedToDict_totaal)
    
    df_stackedToDict_totaal_zwolle = df_copyForStackGrouped_totaal_zwolle.to_dict('list')
    df_stackedToDict_totaal_zwolle['sectoren'] = sectoren_totaal_zwolle

    # Maak er een CDS van
    source_grafiek_totaal_zwolle= ColumnDataSource(df_stackedToDict_totaal_zwolle)

    df_stackedToDict_totaal_flevoland = df_copyForStackGrouped_totaal_flevoland.to_dict('list')
    df_stackedToDict_totaal_flevoland['sectoren'] = sectoren_totaal_flevoland

    # Maak er een CDS van
    source_grafiek_totaal_flevoland = ColumnDataSource(df_stackedToDict_totaal_flevoland)

    # Bereid additionele data voor grafiek voor, zoals legenda en kleurtjes - moet wel *voor* de update functie!
    leg_totaal = [value(x) for x in kolommen_totaal]
    colors_totaal = colortable[0:len(kolommen_totaal)]

    leg_totaal_zwolle = [value(x) for x in kolommen_totaal_zwolle]
    colors_totaal_zwolle = colortable[0:len(kolommen_totaal_zwolle)]

    leg_totaal_flevoland = [value(x) for x in kolommen_totaal_flevoland]
    colors_totaal_flevoland = colortable[0:len(kolommen_totaal_flevoland)]

    #Onderstaande code is om een update uit te voeren, maar werkt nog niet vandaar in commentaar
    #def update_jaar(attr, old, new):
    # Get jaar value
    #   cols = [jaar_select_totaal.value]

    #   df_copyForStack_totaal =  dsc.copy()
    #   df_copyForStackGrouped_totaal = df_copyForStack.groupby(['SECTORUNIT SBB']).sum()[cols]
    #   df_stackedToDict_totaal = df_copyForStackGrouped.to_dict('list')
    #   df_stackedToDict_totaal['sectoren'] = sectoren_totaal
    #   newSource_totaal = ColumnDataSource(df_stackedToDict_totaal)
    #   source_grafiek_totaal.data.update(newSource.data)

    # Update ook gegevens voor legenda
    #   leg = [value(x) for x in cols]
    #   colors = colortable[0:len(cols)] 

    # finally, connect the callbacks to the filter widgets
    sector_select_totaal.js_on_change('value', generic_callback_totaal)
    jaar_select_totaal.js_on_change('value', generic_callback_totaal)

    sector_select_totaal_zwolle.js_on_change('value', generic_callback_totaal_zwolle)
    jaar_select_totaal_zwolle.js_on_change('value', generic_callback_totaal_zwolle)

    sector_select_totaal_flevoland.js_on_change('value', generic_callback_totaal_flevoland)
    jaar_select_totaal_flevoland.js_on_change('value', generic_callback_totaal_flevoland)
    #jaar_select.on_change('value', update_jaar)


    #Teken grafiek
    grafiek_totaal = figure(x_range=sectoren_totaal, plot_height=800, title="Totalen per SBB sector per jaar")
    grafiek_totaal.vbar_stack(kolommen_totaal, x='sectoren', width=0.8, source=source_grafiek_totaal, legend=leg_totaal, color=colors_totaal)

    windesheim_logo = "img/download.png"
    logo_src = ColumnDataSource(dict(url = [windesheim_logo]))

    page_logo = figure(plot_width = 500, plot_height = 500, title="")
    page_logo.toolbar.logo = None
    page_logo.toolbar_location = None
    page_logo.x_range=Range1d(start=0, end=1)
    page_logo.y_range=Range1d(start=0, end=1)
    page_logo.xaxis.visible = None
    page_logo.yaxis.visible = None
    page_logo.xgrid.grid_line_color = None
    page_logo.ygrid.grid_line_color = None
    page_logo.image_url(url='url', x=0.05, y = 0.85, h=0.7, w=0.9, source=logo_src)
    page_logo.outline_line_alpha = 0

    grafiek_totaal.width=900
    grafiek_totaal.y_range.start = 0
    grafiek_totaal.x_range.range_padding = 0.1
    grafiek_totaal.xaxis.major_label_orientation = 'vertical'
    grafiek_totaal.xgrid.grid_line_color = None
    grafiek_totaal.left[0].formatter.use_scientific = False

    grafiek_totaal_zwolle = figure(x_range=sectoren_totaal_zwolle, plot_height=800, title="Totalen per SBB sector per jaar")
    grafiek_totaal_zwolle.vbar_stack(kolommen_totaal_zwolle, x='sectoren', width=0.8, source=source_grafiek_totaal_zwolle, legend=leg_totaal_zwolle, color=colors_totaal_zwolle)

    grafiek_totaal_zwolle.width=900
    grafiek_totaal_zwolle.y_range.start = 0
    grafiek_totaal_zwolle.x_range.range_padding = 0.1
    grafiek_totaal_zwolle.xaxis.major_label_orientation = 'vertical'
    grafiek_totaal_zwolle.xgrid.grid_line_color = None
    grafiek_totaal_zwolle.left[0].formatter.use_scientific = False

    grafiek_totaal_flevoland = figure(x_range=sectoren_totaal_zwolle, plot_height=800, title="Totalen per SBB sector per jaar")
    grafiek_totaal_flevoland.vbar_stack(kolommen_totaal_zwolle, x='sectoren', width=0.8, source=source_grafiek_totaal_flevoland, legend=leg_totaal_flevoland, color=colors_totaal_flevoland)

    grafiek_totaal_flevoland.width=900
    grafiek_totaal_flevoland.y_range.start = 0
    grafiek_totaal_flevoland.x_range.range_padding = 0.1
    grafiek_totaal_flevoland.xaxis.major_label_orientation = 'vertical'
    grafiek_totaal_flevoland.xgrid.grid_line_color = None
    grafiek_totaal_flevoland.left[0].formatter.use_scientific = False

    #layout = row(p, jaren)
    tab1_widgets = column(sector_select_totaal,jaar_select_totaal,data_table_totaal)
    tab1_contents = row(grafiek_totaal, tab1_widgets)
    tab2_widgets = column(sector_select_totaal_zwolle,jaar_select_totaal_zwolle,data_table_totaal_zwolle)
    tab2_contents = row(grafiek_totaal_zwolle, tab2_widgets) 
    tab3_widgets = column(sector_select_totaal_flevoland,jaar_select_totaal_flevoland,data_table_totaal_flevoland)
    tab3_contents = row(grafiek_totaal_flevoland, tab3_widgets)
    tab1 = Panel(child=tab1_contents, title="Totaal")
    tab2 = Panel(child=tab2_contents, title="Zwolle")
    tab3 = Panel(child=tab3_contents, title="Flevoland")
    tabs = Tabs(tabs=[ tab1, tab2, tab3 ])

    doc.title = "Sectorunit SBB chart"
    doc.add_root(tabs)

In [ ]:
apps = {'/': Application(FunctionHandler(make_document))}

server = Server(apps, port=5006, allow_websocket_origin=['localhost:5006','nm-interactive.net:5006'])
server.start()

In [ ]:
server.stop()